In [2]:
pip install psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 246.8 kB/s eta 0:00:03
   ------------------ --------------------- 0.5/1.2 MB 246.8 kB/s eta 0:00:03
   --------------------------- ------------ 0.8/1.2 MB 325.8 kB/s eta 0:00:02
   ------------------------------------ --- 1.0/1.2 MB 449.4 kB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 440.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import psycopg2


# PostgreSQL 연결 테스트
conn = psycopg2.connect(
    host="3.37.207.16",
    port=5432,
    database="postgres",
    user="postgres",
    password="password",
)
print("PostgreSQL 연결 성공!")
conn.close()

PostgreSQL 연결 성공!


## 회귀모델 사용 ver.

In [ ]:
import numpy as np
import joblib
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1. 임베딩 모델 및 벡터DB 로드
embedding_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
)
vector_db = FAISS.load_local(
    "vectorstore/news_db", embedding_model, allow_dangerous_deserialization=True
)

# 2. 이미 훈련된 회귀모델 로드
reg_model = joblib.load("models/regression_model.pkl")  # 경로는 실제 파일 위치로

# 3. 쿼리 입력 및 top-N 검색
query = "미국 금리 인하 시 국내 증시가 어떤 흐름을 보일까?"
top_k = 5
results_with_scores = vector_db.similarity_search_with_score(query, k=top_k)
retrieved_docs = [doc for doc, _ in results_with_scores]

# 4. 각 문서에서 회귀모델 입력 피처 추출
def extract_features(doc):
    # 예시: 문서 메타데이터에서 피처 추출
    meta = doc.metadata
    return [
        meta.get("impact_score", 0),
        meta.get("d_minus_14_close", 0),
        meta.get("d_minus_14_volume", 0),
        # 필요한 추가 피처...
    ]

features = np.array([extract_features(doc) for doc in retrieved_docs])

# 5. 회귀모델로 유사도 예측 및 리랭킹
predicted_scores = reg_model.predict(features)
reranked_indices = np.argsort(predicted_scores)[::-1]
reranked_docs = [retrieved_docs[i] for i in reranked_indices]

# 6. 챗봇 컨텍스트 구성 및 응답 생성
context = "\n\n".join([doc.page_content for doc in reranked_docs])
# 이후 llm_chain 등으로 답변 생성

print("최종 리랭킹 결과:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"{i}. {doc.page_content[:100]} ...")


# 프롬프트 템플릿 정의
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
- 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
- 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
- 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

답변 작성 시 반드시 다음을 지켜주세요:
1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>
검색 결과: {context}
질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

prompt = PromptTemplate(
    input_variables=["context", "question"], template=prompt_template
)
llm = LlamaCpp(
    model_path="...",  # 실제 모델 경로
    n_gpu_layers=0,
    n_ctx=2048,
    n_threads=4,
    verbose=True,
    n_batch=512,
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

context = "\n\n".join([doc.page_content for doc in reranked_docs])
response = llm_chain.run({"context": context, "question": query})

print("🧠 챗봇 응답:\n", response)

## 회귀모델 사용 X, Cross-encoder 사용 ver.

In [ ]:
import numpy as np
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1. 임베딩 모델 및 벡터DB 로드
embedding_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS"
)
vector_db = FAISS.load_local(
    "vectorstore/news_db", embedding_model, allow_dangerous_deserialization=True
)

# 2. 사용자 질문 임베딩 및 Top-K 검색
query = "미국 금리 인상 시 국내 증시에 어떤 영향을 줄까?"
top_k = 10
results_with_scores = vector_db.similarity_search_with_score(query, k=top_k)
retrieved_docs = [doc for doc, _ in results_with_scores]

# 3. Cross-Encoder 리랭커 준비 (예시: BAAI/bge-reranker-v2-m3)
reranker_model_name = "BAAI/bge-reranker-v2-m3"
tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(
    reranker_model_name
).to("cpu")
reranker_model.eval()

# 4. Cross-Encoder로 쿼리-문서 쌍 리랭킹
pairs = [(query, doc.page_content) for doc in retrieved_docs]
inputs = tokenizer(
    [f"{q} [SEP] {d}" for q, d in pairs],
    return_tensors="pt",
    padding=True,
    truncation=True,
)
with torch.no_grad():
    scores = reranker_model(**inputs).logits.squeeze().cpu().numpy()
reranked_indices = np.argsort(scores)[::-1]
reranked_docs = [retrieved_docs[i] for i in reranked_indices]

# 5. LLM 프롬프트 및 답변 생성 (예시)
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = LlamaCpp(
    model_path="/app/ml_models/quantized_llama/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf",
    n_gpu_layers=0,
    n_ctx=2048,
    n_threads=4,
    verbose=True,
    n_batch=512,
)

prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
- 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
- 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
- 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

답변 작성 시 반드시 다음을 지켜주세요:
1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>
검색 결과: {context}
질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
prompt = PromptTemplate(
    input_variables=["context", "question"], template=prompt_template
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

context = "\n\n".join(
    [doc.page_content for doc in reranked_docs[:5]]
)  # 최종 top-5만 사용
response = llm_chain.run({"context": context, "question": query})

print("🧠 챗봇 응답:\n", response)

## rag_pipeline.py (벡터 DB & Cross-encoder)

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from llama_cpp import Llama
from typing import List, Dict

class CSVDummyVectorDB:
    def __init__(self, csv_path: str):
        # CSV 파일 로드 (embedding 컬럼은 문자열 "[0.1, 0.2, ...]" 형태)
        df = pd.read_csv(csv_path)
        self.texts = df['content'].tolist()
        # embedding 컬럼 문자열을 리스트로 변환 후 numpy 배열 생성
        self.embeddings = np.vstack(df['embedding'].apply(eval).to_numpy()).astype('float32')
        # FAISS 인덱스 생성 및 벡터 정규화
        faiss.normalize_L2(self.embeddings)
        self.index = faiss.IndexFlatIP(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def search(self, query_vec: np.ndarray, top_k: int = 10) -> List[str]:
        faiss.normalize_L2(query_vec)
        scores, indices = self.index.search(query_vec, top_k)
        return [self.texts[i] for i in indices[0]]

class NewsTossChatbot:
    def __init__(
        self,
        csv_path: str = "/app/db/news_v2_vector_202506122113.csv",
        embedding_model_name: str = "snunlp/KR-SBERT-V40K-klueNLI-augSTS",
        reranker_model_name: str = "BAAI/bge-reranker-v2-m3",
        llm_model_path: str = "/app/models/quantized_llama3/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf",
        k: int = 10,
        rerank_top_n: int = 5
    ):
        # 1. 임베딩 모델 초기화
        self.embedding_model = SentenceTransformer(embedding_model_name)
        # 2. CSV 기반 벡터 DB 초기화
        self.vector_db = CSVDummyVectorDB(csv_path)
        # 3. Cross-Encoder 리랭커 초기화
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name).to("cpu")
        self.reranker_model.eval()
        # GPU 사용 시(예시, 주석 해제):
        # self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name).to("cuda")
        # 4. LLaMA3 LLM 초기화
        self.llm = Llama(
            model_path=llm_model_path,
            n_ctx=2048,
            n_threads=4,      # CPU 코어 수
            n_gpu_layers=0,   # CPU만 사용
            verbose=False
        )
        # GPU 사용 시(예시, 주석 해제):
        # self.llm = Llama(
        #     model_path=llm_model_path,
        #     n_ctx=2048,
        #     n_threads=4,
        #     n_gpu_layers=8,  # GPU 레이어 수 (환경에 맞게 조정)
        #     verbose=True
        # )
        # 5. 프롬프트 템플릿
        self.prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
        당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
        - 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
        - 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
        - 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

        답변 작성 시 반드시 다음을 지켜주세요:
        1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
        2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
        3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
        4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
        5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        검색 결과: {context}
        질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        self.k = k
        self.rerank_top_n = rerank_top_n

    def retrieve_docs(self, query: str) -> List[str]:
        """CSV+FAISS 기반 벡터DB에서 Top-K 뉴스 검색"""
        query_emb = self.embedding_model.encode([query]).astype('float32')
        docs = self.vector_db.search(query_emb, top_k=self.k)
        return docs

    def rerank_docs(self, query: str, docs: List[str]) -> List[str]:
        """Cross-Encoder로 리랭킹 후 상위 N개 뉴스 반환"""
        pairs = [(query, doc) for doc in docs]
        inputs = self.reranker_tokenizer(
            [f"{q} [SEP] {d}" for q, d in pairs],
            return_tensors="pt",
            padding=True,
            truncation=True
        )
        # CPU 기준
        with torch.no_grad():
            scores = self.reranker_model(**inputs).logits.squeeze().cpu().numpy()
        # GPU 사용 시(예시, 주석 해제):
        # with torch.no_grad():
        #     scores = self.reranker_model(**inputs.to("cuda")).logits.squeeze().cpu().numpy()
        reranked = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
        return [doc for _, doc in reranked[:self.rerank_top_n]]

    def generate_answer(self, context: str, question: str) -> str:
        """LLM 직접 추론"""
        full_prompt = self.prompt_template.format(context=context, question=question)
        output = self.llm(full_prompt, max_tokens=512, stop=["<|eot_id|>"])
        return output['choices'][0]['text']

    def answer(self, query: str) -> Dict[str, str]:
        # 1. 뉴스 검색
        docs = self.retrieve_docs(query)
        # 2. 리랭킹
        reranked_docs = self.rerank_docs(query, docs)
        # 3. 컨텍스트 생성
        context = "\n\n".join(reranked_docs)
        # 4. 답변 생성
        answer = self.generate_answer(context, query)
        return {"answer": answer, "reranked_docs": reranked_docs}


### 테스트!!~!!!

In [2]:
pip install torch

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
    --------------------------------------- 4.5/216.1 MB 22.3 MB/s eta 0:00:10
   - -------------------------------------- 8.9/216.1 MB 22.1 MB/s eta 0:00:10
   -- ------------------------------------- 13.1/216.1 MB 21.6 MB/s eta 0:00:10
   --- ------------------------------------ 17.8/216.1 MB 22.0 MB/s eta 0:00:09
   ---- ----------------------------------- 22.5/216.1 MB 22.3 MB/s eta 0:00:09
   ----- ---------------------------------- 27.5/216.1 MB 22.4 MB/s eta 0:00:09
   ----- ---------------------------------- 31.5/216.1 MB 22.2 MB/s eta 0:00:09
   ------ --------------------------------- 35.4/216.1 MB 21.8 MB/s eta 0:00:09
   ------- -------------------------------- 40.1/216.1 MB 21.8 MB/s eta 0:00:09
   -------- ------------------------------- 45.1/216.1 MB 22.1 MB/s eta 0:00:08
   --------- ------------------------------ 49.8/216.1 MB 22.2 MB/s eta 0:00:08
   ---------- ----------------------------- 55.1/21

In [4]:
pip install sentence-transformers

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -------- ------------------------------- 2.1/10.5 MB 19.7 MB/s eta 0:00:01
   ---------------------- ----------------- 5.8/10.5 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.5 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 16.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 23.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ------------ --------------------------- 4.7/15.0 MB 23.7 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/15.0 MB 23.5 MB/s eta 0:00:01
   -------------------------------------- - 14.4/15.0 MB 23.8 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 23.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/67.9 MB ? eta -:--:--
     -- ------------------------------------- 4.5/67.9 MB 22.4 MB/s eta 0:00:03
     ----- ---------------------------------- 9.7/67.9 MB 24.2 MB/s eta 0:00:03
     -------- ------------------------------ 14.7/67.9 MB 24.3 MB/s eta 0:00:03
     ----------- --------------------------- 19.9/67.9 MB 24.7 MB/s eta 0:00:02
     -------------- ------------------------ 25.7/67.9 MB 25.1 MB/s eta 0:00:02
     ----------------- --------------------- 30.9/67.9 MB 24.9 MB/s eta 0:00:02
     -------------------- ------------------ 35.7/67.9 MB 24.9 MB/s eta 0:00:02
     ----------------------- --------------- 40.9/67.9 MB 24.8 MB/s eta 0:00:02
     -------------------------- ------------ 46.4/67.9 MB 24.8 MB/s eta 0:00:01
     ----------------------------- --------- 51.1/67.9 MB 24.8 MB/s eta 0:00:01
     -------------------------------- ------ 56.1/67.9 MB 24.5 MB/s eta 0:00:01
     ----------------------------------- --- 61.

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.11.4 using CMake 4.0.2 (wheel)
      *** Configuring CMake...
      2025-06-12 21:56:36,530 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\user\AppData\Local\Temp\tmp04rfoju9\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error ori

In [9]:
import os
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from llama_cpp import Llama

# 1. CSV+FAISS 벡터DB 클래스
class CSVDummyVectorDB:
    def __init__(self, csv_path: str):
        df = pd.read_csv(csv_path)
        self.texts = df['content'].tolist()
        self.embeddings = np.vstack(df['embedding'].apply(eval).to_numpy()).astype('float32')
        faiss.normalize_L2(self.embeddings)
        self.index = faiss.IndexFlatIP(self.embeddings.shape[1])
        self.index.add(self.embeddings)

    def search(self, query_vec: np.ndarray, top_k: int = 10):
        faiss.normalize_L2(query_vec)
        scores, indices = self.index.search(query_vec, top_k)
        return [self.texts[i] for i in indices[0]]

# 2. 주요 파라미터 (로컬 환경에 맞게 수정)
csv_path = "./news_v2_vector_202506122113.csv"  # 노트북 파일 위치
embedding_model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
reranker_model_name = "BAAI/bge-reranker-v2-m3"
llm_model_path = "./llama-3-Korean-Bllossom-8B-Q4_K_M.gguf"  # 로컬에 다운받은 모델 경로

# 3. 모델 로딩
embedding_model = SentenceTransformer(embedding_model_name)
vector_db = CSVDummyVectorDB(csv_path)
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name).to("cpu")
reranker_model.eval()
llm = Llama(
    model_path=llm_model_path,
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=0,
    verbose=False
)

# 4. 프롬프트 템플릿
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 주식 투자자를 위한 뉴스 기반 정보 어시스턴트 챗봇 '뉴스토스'입니다.
당신의 임무는 실시간 뉴스와 과거 유사사건 뉴스 데이터를 바탕으로,
- 사용자의 투자 판단에 도움이 되는 정보를 제공하고,
- 과거 유사사건, 해당 시기의 주가 흐름, 관련 리포트의 핵심 내용을 구체적으로 인용하며,
- 미래 전망 질문에는 과거 사례를 근거로 신중하게 의견을 제시하는 것입니다.

답변 작성 시 반드시 다음을 지켜주세요:
1. 답변 내용 중 포함되는 과거 유사사건의 날짜, 사건명, 당시 주가 흐름(상승/하락/횡보 등), 주요 리포트 내용은 구체적으로 인용하세요.
2. 미래 전망 질문에는 과거 유사사건을 근거로 논리적인 전망을 제시하세요.
3. 답변 마지막에는 '⭐️투자 판단은 본인의 책임입니다.⭐️'라는 안내문을 추가하세요.
4. 답변은 반드시 한글로, 명확하고 간결하게 작성하세요.
5. 제공된 검색 결과(유사도 높은 과거 뉴스, 주가 데이터, 리포트 등)만 근거로 사용하세요. 근거가 없으면 '근거가 없는데 답변해도 될까? 이건 너의 소중한 돈이 걸린 문제야'라고 하세요.
<|eot_id|><|start_header_id|>user<|end_header_id|>
검색 결과: {context}
질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

# 5. 전체 파이프라인 함수
def rag_answer(question, top_k=10, rerank_n=5):
    # 1. 임베딩
    query_emb = embedding_model.encode([question]).astype('float32')
    # 2. FAISS 검색
    docs = vector_db.search(query_emb, top_k=top_k)
    # 3. Cross-Encoder 리랭킹
    pairs = [(question, doc) for doc in docs]
    inputs = reranker_tokenizer(
        [f"{q} [SEP] {d}" for q, d in pairs],
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    with torch.no_grad():
        scores = reranker_model(**inputs).logits.squeeze().cpu().numpy()
    reranked = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
    reranked_docs = [doc for _, doc in reranked[:rerank_n]]
    # 4. LLM 답변 생성
    context = "\n\n".join(reranked_docs)
    full_prompt = prompt_template.format(context=context, question=question)
    output = llm(full_prompt, max_tokens=512, stop=["<|eot_id|>"])
    return {
        "answer": output['choices'][0]['text'],
        "reranked_docs": reranked_docs
    }

# 6. 사용 예시
question = "금리인하했는데 주가에 미치는 영향은?"
result = rag_answer(question)
print("답변:", result["answer"])
print("참고 뉴스:", result["reranked_docs"])


ModuleNotFoundError: No module named 'llama_cpp'